# Predicción de Default usando SVC

Este notebook implementa un modelo de Support Vector Classifier (SVC) para predecir el default de tarjetas de crédito.

## Contenido:
1. Importación de librerías
2. Funciones de limpieza de datos
3. Definición del modelo y pipeline
4. Optimización de hiperparámetros
5. Funciones de evaluación
6. Carga y procesamiento de datos
7. Entrenamiento y evaluación del modelo

In [1]:
# Importación de librerías necesarias
import zipfile
import pickle
import gzip
import json
import os
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import balanced_accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

## 1. Función de Limpieza de Datos

La función `clean_data` prepara los datos eliminando columnas innecesarias, renombrando variables y aplicando filtros específicos.

In [2]:
def clean_data(df):
    """
    Limpia y preprocesa el dataset de tarjetas de crédito.
    
    Args:
        df (pd.DataFrame): Dataset original
        
    Returns:
        pd.DataFrame: Dataset limpio
    """
    # Creamos una copia
    df = df.copy()
    
    # Quitamos la columna ID
    df = df.drop('ID', axis=1)
    
    # Renombramos la columna target
    df = df.rename(columns={'default payment next month': 'default'})
    
    # Eliminamos registros con datos faltantes
    df = df.dropna()
    
    # Eliminar filas donde el registro es NA en variables categóricas importantes
    df = df[(df['EDUCATION'] != 0 ) & (df['MARRIAGE'] != 0)]
    
    # Para la columna EDUCATION, valores > 4 indican niveles superiores de educación
    # Agrupamos estos valores en la categoría "others" (4)
    df.loc[df['EDUCATION'] > 4, 'EDUCATION'] = 4

    return df

## 2. Definición del Pipeline del Modelo

Se define un pipeline que incluye preprocesamiento, reducción de dimensionalidad y clasificación.

In [3]:
def model():
    """
    Crea y retorna el pipeline del modelo de machine learning.
    
    Returns:
        Pipeline: Pipeline con preprocesamiento y clasificador SVC
    """
    # Variables categóricas para OneHotEncoding
    categories = ['SEX', 'EDUCATION', 'MARRIAGE']  
    
    # Variables numéricas para estandarización
    numerics = [
        "LIMIT_BAL", "AGE", "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6",
        "BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6", 
        "PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5","PAY_AMT6"
    ]

    # Preprocesamiento de variables
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown='ignore'), categories),
            ('scaler', StandardScaler(), numerics)
        ],
        remainder='passthrough'
    )

    # Selección de características más relevantes
    selectkbest = SelectKBest(score_func=f_classif)

    # Pipeline completo
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('pca', PCA()),  # Reducción de dimensionalidad
        ("selectkbest", selectkbest),
        ('classifier', SVC(kernel='rbf', random_state=42))
    ])

    return pipeline

## 3. Optimización de Hiperparámetros

Función para optimizar los hiperparámetros del modelo usando GridSearchCV.

In [4]:
def hyperparameters(model, n_splits, x_train, y_train, scoring):
    """
    Optimiza los hiperparámetros del modelo usando validación cruzada.
    
    Args:
        model: Pipeline del modelo
        n_splits (int): Número de folds para validación cruzada
        x_train: Características de entrenamiento
        y_train: Variable objetivo de entrenamiento
        scoring (str): Métrica de evaluación
        
    Returns:
        GridSearchCV: Modelo optimizado
    """
    # Búsqueda de parámetros con validación cruzada
    estimator = GridSearchCV(
        estimator=model,
        param_grid = {
            'pca__n_components': [20, 21],      # Componentes principales
            'selectkbest__k': [12],             # Número de características seleccionadas
            'classifier__kernel': ['rbf'],      # Tipo de kernel
            'classifier__gamma': [0.099]        # Parámetro para el kernel RBF
        },
        cv=n_splits,
        refit=True,
        verbose=0,
        return_train_score=False,
        scoring=scoring
    )
    
    # Entrenamos el modelo con los mejores parámetros
    estimator.fit(x_train, y_train)

    return estimator

## 4. Funciones de Evaluación del Modelo

Funciones para calcular métricas de rendimiento y matrices de confusión.

In [5]:
def metrics(model, x_train, y_train, x_test, y_test):
    """
    Calcula métricas de rendimiento para conjuntos de entrenamiento y prueba.
    
    Args:
        model: Modelo entrenado
        x_train, y_train: Datos de entrenamiento
        x_test, y_test: Datos de prueba
        
    Returns:
        tuple: Métricas de entrenamiento y prueba
    """
    # Realizamos las predicciones
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Métricas para conjunto de entrenamiento
    train_metrics = {
        'type': 'metrics',
        'dataset': 'train',
        'precision': precision_score(y_train, y_train_pred, average='binary'),
        'balanced_accuracy': balanced_accuracy_score(y_train, y_train_pred),
        'recall': recall_score(y_train, y_train_pred, average='binary'),
        'f1_score': f1_score(y_train, y_train_pred, average='binary')
    }

    # Métricas para conjunto de prueba
    test_metrics = {
        'type': 'metrics',
        'dataset': 'test',
        'precision': precision_score(y_test, y_test_pred, average='binary'),
        'balanced_accuracy': balanced_accuracy_score(y_test, y_test_pred),
        'recall': recall_score(y_test, y_test_pred, average='binary'),
        'f1_score': f1_score(y_test, y_test_pred, average='binary')
    }

    return train_metrics, test_metrics

In [6]:
def matrix(model, x_train, y_train, x_test, y_test):
    """
    Calcula las matrices de confusión para conjuntos de entrenamiento y prueba.
    
    Args:
        model: Modelo entrenado
        x_train, y_train: Datos de entrenamiento
        x_test, y_test: Datos de prueba
        
    Returns:
        tuple: Matrices de confusión de entrenamiento y prueba
    """
    # Realizamos las predicciones
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Calculamos los valores de la matriz de confusión
    cm_train = confusion_matrix(y_train, y_train_pred)
    tn_train, fp_train, fn_train, tp_train = cm_train.ravel()

    cm_test = confusion_matrix(y_test, y_test_pred)
    tn_test, fp_test, fn_test, tp_test = cm_test.ravel()

    # Matriz de confusión para entrenamiento
    train_matrix = {
        'type': 'cm_matrix',
        'dataset': 'train', 
        'true_0': {
            'predicted_0': int(tn_train),
            'predicted_1': int(fp_train)
        },
        'true_1': {
            'predicted_0': int(fn_train),
            'predicted_1': int(tp_train)
        }
    }

    # Matriz de confusión para prueba
    test_matrix = {
        'type': 'cm_matrix',
        'dataset': 'test', 
        'true_0': {
            'predicted_0': int(tn_test),
            'predicted_1': int(fp_test)
        },
        'true_1': {
            'predicted_0': int(fn_test),
            'predicted_1': int(tp_test)
        }
    }

    return train_matrix, test_matrix

## 5. Funciones de Guardado

Funciones para guardar el modelo entrenado y las métricas calculadas.

In [7]:
def save_model(model):
    """
    Guarda el modelo entrenado en formato comprimido.
    
    Args:
        model: Modelo entrenado a guardar
    """
    # Crear las carpetas si no existen
    os.makedirs('files/models', exist_ok=True)

    with gzip.open('files/models/model.pkl.gz', 'wb') as f:
        pickle.dump(model, f)


def save_metrics(metrics):
    """
    Guarda las métricas en formato JSON.
    
    Args:
        metrics (list): Lista de diccionarios con métricas
    """
    # Crear las carpetas si no existen
    os.makedirs('files/output', exist_ok=True)

    with open("files/output/metrics.json", "w") as f:
        for metric in metrics:
            json_line = json.dumps(metric)
            f.write(json_line + "\n")

## 6. Ejecución Principal

### Carga y Preprocesamiento de Datos

In [8]:
# Verificar y establecer el directorio de trabajo
import os
print(f"Directorio actual: {os.getcwd()}")

# Cambiar al directorio del proyecto si es necesario
project_dir = r'c:\Users\david\LAB-11-prediccion-del-default-usando-svc-DavidZapataN'
if os.getcwd() != project_dir:
    os.chdir(project_dir)
    print(f"Cambiado a: {os.getcwd()}")

# Nombres de los archivos de datos
file_Test = 'files/input/test_data.csv.zip'
file_Train = 'files/input/train_data.csv.zip'

# Verificar que los archivos existen
print(f"Archivo test existe: {os.path.exists(file_Test)}")
print(f"Archivo train existe: {os.path.exists(file_Train)}")

# Lectura del archivo de prueba
with zipfile.ZipFile(file_Test, 'r') as zip:
    with zip.open('test_default_of_credit_card_clients.csv') as f:
        df_Test = pd.read_csv(f)

# Lectura del archivo de entrenamiento
with zipfile.ZipFile(file_Train, 'r') as zip:
    with zip.open('train_default_of_credit_card_clients.csv') as f:
        df_Train = pd.read_csv(f)

print(f"Dataset de prueba cargado: {df_Test.shape}")
print(f"Dataset de entrenamiento cargado: {df_Train.shape}")

Directorio actual: c:\Users\david\LAB-11-prediccion-del-default-usando-svc-DavidZapataN\homework
Cambiado a: c:\Users\david\LAB-11-prediccion-del-default-usando-svc-DavidZapataN
Archivo test existe: True
Archivo train existe: True
Dataset de prueba cargado: (9000, 25)
Dataset de entrenamiento cargado: (21000, 25)


In [9]:
# Limpieza de los datasets
df_Test = clean_data(df_Test)
df_Train = clean_data(df_Train)

# División de los datasets en características (X) y variable objetivo (y)
x_train, y_train = df_Train.drop('default', axis=1), df_Train['default']
x_test, y_test = df_Test.drop('default', axis=1), df_Test['default']

### Entrenamiento y Optimización del Modelo

In [10]:
# Creación del pipeline del modelo
model_pipeline = model()

# Optimización de hiperparámetros con validación cruzada
model_pipeline = hyperparameters(model_pipeline, 10, x_train, y_train, 'balanced_accuracy')

### Evaluación y Guardado de Resultados

In [11]:
# Guardado del modelo entrenado
save_model(model_pipeline)

# Cálculo de métricas de rendimiento
train_metrics, test_metrics = metrics(model_pipeline, x_train, y_train, x_test, y_test)

# Cálculo de matrices de confusión
train_matrix, test_matrix = matrix(model_pipeline, x_train, y_train, x_test, y_test)

# Guardado de todas las métricas calculadas
save_metrics([train_metrics, test_metrics, train_matrix, test_matrix])